# Data import for Schiller_2020/Mayr datasets:

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from anndata import AnnData, concat

In [2]:
adata = sc.read("../../../data/HLCA_extended/extension_datasets/raw/Schiller/munich_cohort_human_dataset.h5ad")

/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [3]:
adata_raw = AnnData(X=adata.layers["counts"], obs=adata.obs, var=adata.var)
adata_raw

AnnData object with n_obs × n_vars = 38648 × 32104
    obs: 'patient_id', 'n_counts', 'n_genes', 'percent.mito', 'louvain', 'health_status', 'cell_type', 'cell_state_label', 'tissue_label'
    var: 'n_cells', 'highly_variable', 'n_counts'

# Ensure consistent naming

In [4]:
adata_raw.obs.rename(columns={"patient_id": "subject_ID",
                              "health_status": "condition",
                              "cell_state_label": "original_celltype_ann",
                              "cell_type": "low_res_cell_type"}, inplace=True)

In [5]:
adata_raw.obs["sample"] = adata_raw.obs.subject_ID
adata_raw.obs["study"] = "Schiller2020"
adata_raw.obs["dataset"] = adata_raw.obs.study

# Remove unnecessary obs columns

In [6]:
adata_raw.obs.drop(columns=["n_counts", "n_genes", "percent.mito", "louvain", "tissue_label"], inplace=True)
adata_raw.var.drop(columns=["n_cells", "highly_variable", "n_counts"], inplace=True)

# Add age & sex & disease

In [7]:
def add_age (row):
   if row['subject_ID'] == "muc10380":
      return '73'
   if row['subject_ID'] == "muc10381":
      return '51'
   if row['subject_ID'] == "muc3843":
      return '63'
   if row['subject_ID'] == "muc4658":
      return '59'
   if row['subject_ID'] == "muc4659":
      return '75'
   if row['subject_ID'] == "muc5103":
      return '60'
   if row['subject_ID'] == "muc5104":
      return '76'
   if row['subject_ID'] == "muc5105":
      return '65'
   if row['subject_ID'] == "muc5212":
      return '62'
   if row['subject_ID'] == "muc5213":
      return '23'
   if row['subject_ID'] == "muc5288":
      return '76'
   if row['subject_ID'] == "muc5289":
      return '40'
   if row['subject_ID'] == "muc8257":
      return '40'
   if row['subject_ID'] == "muc8258":
      return '40'
   if row['subject_ID'] == "muc9826":
      return '57'
   if row['subject_ID'] == "muc9832":
      return '81'
   if row['subject_ID'] == "muc9833":
      return '52'
   return 'undefined'

adata_raw.obs['age'] = adata_raw.obs.apply (lambda row: add_age(row), axis=1)

In [8]:
def add_sex (row):
   if row['subject_ID'] == "muc10380":
      return 'M'
   if row['subject_ID'] == "muc10381":
      return 'M'
   if row['subject_ID'] == "muc3843":
      return 'M'
   if row['subject_ID'] == "muc4658":
      return 'M'
   if row['subject_ID'] == "muc4659":
      return 'F'
   if row['subject_ID'] == "muc5103":
      return 'M'
   if row['subject_ID'] == "muc5104":
      return 'M'
   if row['subject_ID'] == "muc5105":
      return 'M'
   if row['subject_ID'] == "muc5212":
      return 'M'
   if row['subject_ID'] == "muc5213":
      return 'F'
   if row['subject_ID'] == "muc5288":
      return 'M'
   if row['subject_ID'] == "muc5289":
      return 'F'
   if row['subject_ID'] == "muc8257":
      return 'F'
   if row['subject_ID'] == "muc8258":
      return 'F'
   if row['subject_ID'] == "muc9826":
      return 'M'
   if row['subject_ID'] == "muc9832":
      return 'F'
   if row['subject_ID'] == "muc9833":
      return 'F'
   return 'undefined'

adata_raw.obs['sex'] = adata_raw.obs.apply (lambda row: add_sex(row), axis=1)

In [9]:
def add_disease (row):
   if row['subject_ID'] == "muc10380":
      return 'Donor'
   if row['subject_ID'] == "muc10381":
      return 'IPF'
   if row['subject_ID'] == "muc3843":
      return 'Donor'
   if row['subject_ID'] == "muc4658":
      return 'Donor'
   if row['subject_ID'] == "muc4659":
      return 'Donor'
   if row['subject_ID'] == "muc5103":
      return 'Donor'
   if row['subject_ID'] == "muc5104":
      return 'Donor'
   if row['subject_ID'] == "muc5105":
      return 'Donor'
   if row['subject_ID'] == "muc5212":
      return 'Donor'
   if row['subject_ID'] == "muc5213":
      return 'Donor'
   if row['subject_ID'] == "muc5288":
      return 'Donor'
   if row['subject_ID'] == "muc5289":
      return 'COPD'
   if row['subject_ID'] == "muc8257":
      return 'EAA'
   if row['subject_ID'] == "muc8258":
      return 'EAA'
   if row['subject_ID'] == "muc9826":
      return 'IPF'
   if row['subject_ID'] == "muc9832":
      return 'Donor'
   if row['subject_ID'] == "muc9833":
      return 'Donor'
   return 'undefined'

adata_raw.obs['disease'] = adata_raw.obs.apply (lambda row: add_disease(row), axis=1)

# Remove duplicated subject

MLT211 was harvested twice. Hence, muc8257 and muc8258 match to the same subject.    
subject_ID -> 8257 both
sample -> 8257_1 8257_2 

In [10]:
adata_raw.obs.subject_ID.replace({"muc8258": "muc8257"}, inplace=True)

In [11]:
adata_raw.obs["sample"].replace({"muc8257": "muc8257_1",
                                 "muc8258": "muc8257_2"}, inplace=True)

# Subset to 2000 HVGs

In [12]:
def subset_and_pad_adata(adata, gene_set):
    """
    This function uses a gene list provided as a Pandas dataframe with gene symbols and
    Ensembl IDs and subsets a larger Anndata object to only the genes in this list. If
    Not all genes are found in the AnnData object, then zero-padding is performed.
    """
    # Example inputs:
    # genes_filename = '/storage/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/genes_for_mapping.csv'
    # data_filename = '/storage/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/ready/adams.h5ad'
    # gene_set = pd.read_csv(genes_filename)
    # adata = sc.read(data_filename)

    # Prep objects
    if 'gene_symbols' in gene_set.columns:
        gene_set.index = gene_set['gene_symbols']

    else:
        raise ValueError('The input gene list was not of the expected type!\n'
                         'Gene symbols and ensembl IDs are expected in column names:\n'
                         '\t`gene_symbols` and `Unnamed: 0`')

    # Subset adata object
    common_genes = [gene for gene in gene_set['gene_symbols'].values if gene in adata.var_names]
    if len(common_genes) == 0:
        print("WARNING: YOU SHOULD PROBABLY SWITCH YOUR ADATA.VAR INDEX COLUMN TO GENE NAMES"
                  " RATHER THAN IDS! No genes were recovered.")
        return

    adata_sub = adata[:,common_genes].copy()

    # Pad object with 0 genes if needed
    if len(common_genes) < len(gene_set):
        diff = len(gene_set) - len(common_genes)
        print(f'not all genes were recovered, filling in 0 counts for {diff} missing genes...')
        
        # Genes to pad with
        genes_to_add = set(gene_set['gene_symbols'].values).difference(set(adata_sub.var_names))
        new_var = gene_set.loc[genes_to_add]
        
        if 'Unnamed: 0' in new_var.columns:
            # Assumes the unnamed column are ensembl values
            new_var['ensembl'] = new_var['Unnamed: 0']
            del new_var['Unnamed: 0']

        df_padding = pd.DataFrame(data=np.zeros((adata_sub.shape[0],len(genes_to_add))), index=adata_sub.obs_names, columns=new_var.index)
        adata_padding = sc.AnnData(df_padding, var=new_var)
        
        # Concatenate object
        adata_sub = concat([adata_sub, adata_padding], axis=1, join='outer', index_unique=None, merge='unique')

    # Ensure ensembl IDs are available
    adata_sub.var['ensembl'] = gene_set['Unnamed: 0']

    return adata_sub

In [13]:
gene_set = pd.read_csv("genes_for_mapping.csv")
adata_raw_subsetted = subset_and_pad_adata(adata_raw, gene_set)

not all genes were recovered, filling in 0 counts for 45 missing genes...


# Write out object

In [14]:
adata_raw.write("../../../data/HLCA_extended/extension_datasets/ready/full/mayr.h5ad")

/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'sample' as categorical
/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'study' as categorical
/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` param

In [15]:
adata_raw_subsetted.write("../../../data/HLCA_extended/extension_datasets/ready/subsetted/mayr_sub.h5ad")

/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'sample' as categorical
/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'study' as categorical
/home/icb/lukas.heumos/miniconda3/envs/single-cell-analysis/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` param